<a href="https://colab.research.google.com/github/mxngjxa/learning_equality_recommendation/blob/main/learning_equality_v1_2_sparse_topn_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#drive mounting for timestamping
from google.colab import drive
drive.mount('/content/drive')
import os

working_directory = "/content/drive/MyDrive/Colab Notebooks/DAS 435 Jacky/project_2/"
data_directory = os.path.join(working_directory, 'data/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import pandas as pd
import numpy as np


In [42]:
topics = pd.read_csv(data_directory + 'topics.csv')
correlations = pd.read_csv(data_directory + 'correlations.csv')
content = pd.read_csv(data_directory + 'content.csv')

In [43]:
#creating mapping dictionaries for parent and title
title_map = topics.set_index("id")["title"].to_dict()
parent_map = topics.set_index("id")["parent"].to_dict()

#addign parent title columns
topics["parent_title"] = topics["parent"].apply(lambda x: title_map.get(x, ""))
print(topics.shape)

#addding grandpa title
topics["grandpa"] = topics["parent"].apply(lambda x: parent_map.get(x))
topics["grandpa_title"] = topics["grandpa"].apply(lambda x: title_map.get(x, ""))
print(topics.shape)

#adding great grandpa title
topics["ggrandpa"] = topics["grandpa"].apply(lambda x: parent_map.get(x))
topics["ggrandpa_title"] = topics["ggrandpa"].apply(lambda x: title_map.get(x, ""))
print(topics.shape)

#fill in missing parent values
topics["parent"] = topics["parent"].fillna(topics["id"])


(76972, 10)
(76972, 12)
(76972, 14)


In [46]:
content["t"] = content["title"].fillna("") + " | " + content["kind"].fillna("") + " | " + content["description"].fillna("")

topics["t"] = topics["title"] + " @ " + topics["parent_title"] + " @ " + topics["grandpa_title"] + " @ " + topics["ggrandpa_title"] + " | " + topics["description"].fillna("")


In [79]:
topics_en = topics[topics['language'] == 'en'][:5000]
print(topics_en.shape)
topics_en.head()

(5000, 15)


,id,title,description,channel,category,level,language,parent,has_content,parent_title,grandpa,grandpa_title,ggrandpa,ggrandpa_title,t
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False,Junior High Level 3,t_a7bda143557e,Ghana JHS Curriculum (in progress),NaN,,Unit 3.3 Enlargements and Similarities @ Junio...
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True,Flow Charts: Logical Thinking?,t_acbbd893e6af,Engineering,t_9c5f09334d83,MIT Blossoms,Transcripts @ Flow Charts: Logical Thinking? @...
6,t_0008a1bd84ba,12. 20: Bird Reproduction,NaN,ebc86c,supplemental,5,en,t_c44ac9711007,True,12: Vertebrates,t_9be861f67172,Book: Introductory Biology,t_ee558ee40155,Introductory and General Biology,12. 20: Bird Reproduction @ 12: Vertebrates @ ...
7,t_000c0c854f0b,Nepal,NaN,f2e966,source,1,en,t_f9c4ef0d6290,False,Touchable Earth (en),NaN,,None,,Nepal @ Touchable Earth (en) @ @ |
8,t_000d1fb3f2f5,2.1.2 - Logarithms,NaN,e77b55,aligned,5,en,t_b897d168db90,True,2.1 - Exponents and Logarithms,t_fa5b4d3f322b,Unit 2 - Exponential and Logarithmic Functions,t_b1104358008a,Mathematics,2.1.2 - Logarithms @ 2.1 - Exponents and Logar...


In [80]:
correlations

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [88]:
en_corr = correlations[correlations['topic_id'].isin(topics_en['id'])]
print(en_corr.shape)
en_corr

(3896, 2)


,topic_id,content_ids
2,t_00069b63a70a,c_11a1dc0bfb99
5,t_0008a1bd84ba,c_7ff92a954a3d c_8790b074383e
6,t_000d1fb3f2f5,c_07f1d0eec4b2 c_15a6fb858696 c_175e9db3fc44 c...
8,t_00102869fbcb,c_005e793bd0c5 c_066737fa5146 c_08b2070f92e0 c...
10,t_0012a45fa09c,c_dde078b8ea7a
...,...,...
8513,t_23de42006e69,c_41fcbcb41a67 c_50533d1625da c_676bf032381a c...
8514,t_23de8fb175c4,c_86014477b8c2 c_c77ea3e83622
8518,t_23e2be49b89c,c_1c4c2bcc366d c_5c4dc6c6311e c_61f9bc065f7c
8520,t_23e600157719,c_3685e071f1e1 c_3a41730459f0


In [89]:
topics_en = topics_en[topics_en['id'].isin(en_corr['topic_id'])]
print(topics_en.shape)
topics_en

(3896, 15)


,id,title,description,channel,category,level,language,parent,has_content,parent_title,grandpa,grandpa_title,ggrandpa,ggrandpa_title,t
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True,Flow Charts: Logical Thinking?,t_acbbd893e6af,Engineering,t_9c5f09334d83,MIT Blossoms,Transcripts @ Flow Charts: Logical Thinking? @...
6,t_0008a1bd84ba,12. 20: Bird Reproduction,NaN,ebc86c,supplemental,5,en,t_c44ac9711007,True,12: Vertebrates,t_9be861f67172,Book: Introductory Biology,t_ee558ee40155,Introductory and General Biology,12. 20: Bird Reproduction @ 12: Vertebrates @ ...
8,t_000d1fb3f2f5,2.1.2 - Logarithms,NaN,e77b55,aligned,5,en,t_b897d168db90,True,2.1 - Exponents and Logarithms,t_fa5b4d3f322b,Unit 2 - Exponential and Logarithmic Functions,t_b1104358008a,Mathematics,2.1.2 - Logarithms @ 2.1 - Exponents and Logar...
10,t_00102869fbcb,Triangles and polygons,Learning outcomes: students must be able to so...,a91e32,aligned,3,en,t_039cecc12bb8,True,"Space, shape and measurement",t_5a5b745edd47,Form 1,t_bff0c73af728,Malawi Mathematics Syllabus,"Triangles and polygons @ Space, shape and meas..."
13,t_0012a45fa09c,Quiz: materials and techniques,NaN,2ee29d,aligned,4,en,t_6957d4a9f469,True,Creating & conserving,t_40c0c3926927,Special topics in art history,t_1d812a8b00f4,Arts and humanities,Quiz: materials and techniques @ Creating & co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10694,t_23de42006e69,Breastfeeding Positions,NaN,73843d,source,3,en,t_bf93b991bb2a,True,Breastfeeding,t_c14a03a92102,Health,t_c4e849cc09ad,Sikana (English),Breastfeeding Positions @ Breastfeeding @ Heal...
10695,t_23de8fb175c4,Free energy,Unsure whether a chemical reaction will be spo...,2ee29d,aligned,4,en,t_64e268e2d33a,True,Energy and enzymes,t_9ecbc0969706,Biology,t_cc2c9ee75232,Science & Engineering,Free energy @ Energy and enzymes @ Biology @ S...
10700,t_23e2be49b89c,Explore,NaN,ef93f0,supplemental,5,en,t_387c1eaffc44,True,Assessments,t_4678b1196d08,Types of muscles,t_12a18d79499b,12. The Muscular System and Digestive System i...,Explore @ Assessments @ Types of muscles @ 12....
10702,t_23e600157719,The determinant of a 2x2 matrix,Learn what the determinant of a matrix is and ...,0ec697,source,5,en,t_8f11e4984283,True,Matrices,t_baa1c12cc791,Precalculus,t_32e89ff4ce0c,High school,The determinant of a 2x2 matrix @ Matrices @ P...


In [90]:
en_corr.loc[:, "content_ids"] = en_corr["content_ids"].apply(lambda x: x.split())

In [91]:
corr_df = en_corr.explode('content_ids').rename(columns={"content_ids": "content_id"})
corr_df

,topic_id,content_id
2,t_00069b63a70a,c_11a1dc0bfb99
5,t_0008a1bd84ba,c_7ff92a954a3d
5,t_0008a1bd84ba,c_8790b074383e
6,t_000d1fb3f2f5,c_07f1d0eec4b2
6,t_000d1fb3f2f5,c_15a6fb858696
...,...,...
8518,t_23e2be49b89c,c_5c4dc6c6311e
8518,t_23e2be49b89c,c_61f9bc065f7c
8520,t_23e600157719,c_3685e071f1e1
8520,t_23e600157719,c_3a41730459f0


In [108]:
topics_en = topics_en.dropna(subset=['t'])
topics_en = topics_en[['id', 't']]
topics_en

In [109]:
topics_en

,id,t
3,t_00069b63a70a,Transcripts @ Flow Charts: Logical Thinking? @...
6,t_0008a1bd84ba,12. 20: Bird Reproduction @ 12: Vertebrates @ ...
8,t_000d1fb3f2f5,2.1.2 - Logarithms @ 2.1 - Exponents and Logar...
10,t_00102869fbcb,"Triangles and polygons @ Space, shape and meas..."
13,t_0012a45fa09c,Quiz: materials and techniques @ Creating & co...
...,...,...
10694,t_23de42006e69,Breastfeeding Positions @ Breastfeeding @ Heal...
10695,t_23de8fb175c4,Free energy @ Energy and enzymes @ Biology @ S...
10700,t_23e2be49b89c,Explore @ Assessments @ Types of muscles @ 12....
10702,t_23e600157719,The determinant of a 2x2 matrix @ Matrices @ P...


In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def tf_idf_matrix(df):
    """
    Finds the most similar documents to a given document ID based on cosine similarity of TF-IDF vectors.

    Parameters:
        topics (pd.DataFrame): DataFrame containing the 'id' column as document IDs.
        topics_en (pd.DataFrame): DataFrame containing the 't' column with document texts.
        doc_id (int): The document ID for which to find similar documents.
        top_n (int): Number of most similar documents to return. Default is 5.

    Returns:
        pd.DataFrame: A DataFrame containing the top_n similar document IDs and their similarity scores.
    """
    # Combine the datasets on index for alignment
    combined_data = df[['id', 't']].set_index('id')
    #print(combined_data.head())

    # Vectorize the text using TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(combined_data['t'])
    corpus = tfidf_vectorizer.transform(combined_data['t'])

    return tfidf_vectorizer, tfidf_matrix, corpus

vectorizer, matrix, corpus = tf_idf_matrix(df=topics_en)


In [121]:
vectorizer.transform(['hello'])

<1x6967 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [123]:
import time

brute_force = list()

st = time.time()

for vec in corpus:
  brute_force.append(cosine_similarity(matrix, vec))

et = time.time()

print(f"Cosine Similarity (all docs) Runtime: {et - st} seconds")


Cosine Similarity (all docs) Runtime: 9.43246841430664 seconds


In [124]:
!pip install sparse_dot_topn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 4.8 MB/s eta 0:00:00


In [133]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sparse
from sparse_dot_topn import sp_matmul_topn
import time

def tf_idf_matrix(df):
    """
    Vectorizes the text using TF-IDF.
    """
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['t'])
    return tfidf_vectorizer, tfidf_matrix

# DataFrame
df = pd.DataFrame(topics_en)

# Create TF-IDF matrix
vectorizer, tfidf_matrix = tf_idf_matrix(df)

# SKLearn Brute-Force Cosine Similarity
start_sklearn = time.time()
brute_force_similarities = cosine_similarity(tfidf_matrix)
end_sklearn = time.time()

print(f"SKLearn Runtime: {end_sklearn - start_sklearn} seconds")

# Sparse Dot Top-N Cosine Similarity
start_sparse_topn = time.time()
top_n = 2
threshold = 0.0  # Set a threshold if needed

# Compute top-N cosine similarities
sparse_topn_similarities = sp_matmul_topn(
    tfidf_matrix,
    tfidf_matrix.T,
    top_n=top_n,
    threshold=threshold,
    n_threads=2
)
end_sparse_topn = time.time()

print(f"Sparse Dot Top-N Runtime: {end_sparse_topn - start_sparse_topn} seconds")

sklearn_runtime = end_sklearn - start_sklearn
sparse_runtime = end_sparse_topn - start_sparse_topn

print(f"Using the sparse_dot_topnn package is {sklearn_runtime/sparse_runtime:.3} times or {sklearn_runtime/sparse_runtime:.0%} faster")


SKLearn Runtime: 0.2623443603515625 seconds
Sparse Dot Top-N Runtime: 0.05303215980529785 seconds
Using the sparse_dot_topnn package is 4.95 times or 495% faster


In [136]:
topics_main = topics.dropna(subset=['t'])
topics_main = topics_main[['id', 't']]
topics_main

,id,t
0,t_00004da3a1b2,Откриването на резисторите @ Открития и проект...
1,t_000095e03056,Unit 3.3 Enlargements and Similarities @ Junio...
2,t_00068291e9a4,Entradas e saídas de uma função @ Álgebra: fun...
3,t_00069b63a70a,Transcripts @ Flow Charts: Logical Thinking? @...
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...
...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions @ Unit 4 - Relations an...
76968,t_fffbe1d5d43c,Inscribed shapes problem solving @ Miduara @ C...
76969,t_fffe14f1be1e,Lección 7 @ Unidad 4 @ Español Actividades 1ro...
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...


What about on larger datasets?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sparse
from sparse_dot_topn import sp_matmul_topn
import time

def tf_idf_matrix(df):
    """
    Vectorizes the text using TF-IDF.
    """
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['t'])
    return tfidf_vectorizer, tfidf_matrix

# DataFrame
df = pd.DataFrame(topics_en)

# Create TF-IDF matrix
vectorizer, tfidf_matrix = tf_idf_matrix(topics_main)

# SKLearn Brute-Force Cosine Similarity
start_sklearn = time.time()
brute_force_similarities = cosine_similarity(tfidf_matrix)
end_sklearn = time.time()

print(f"SKLearn Runtime: {end_sklearn - start_sklearn} seconds")

# Sparse Dot Top-N Cosine Similarity
start_sparse_topn = time.time()
top_n = 2
threshold = 0.0  # Set a threshold if needed

# Compute top-N cosine similarities
sparse_topn_similarities = sp_matmul_topn(
    tfidf_matrix,
    tfidf_matrix.T,
    top_n=top_n,
    threshold=threshold,
    n_threads=2
)
end_sparse_topn = time.time()

print(f"Sparse Dot Top-N Runtime: {end_sparse_topn - start_sparse_topn} seconds")

sklearn_runtime = end_sklearn - start_sklearn
sparse_runtime = end_sparse_topn - start_sparse_topn

print(f"Using the sparse_dot_topnn package is {sklearn_runtime/sparse_runtime:.3} times or {sklearn_runtime/sparse_runtime:.0%} faster")
